## Business Understanding
### Objective: Build an NLP model that predicts the sentiment (positive, negative, or neutral) of Tweets about Apple and Google products.

### Key Questions
- What is the distribution of sentiments in the dataset?
- How do Apple and Google product-related Tweets differ in sentiment?
- Can we improve customer understanding of the product sentiment trends?

## Data Understanding
Dataset Overview:
The dataset contains over 9,000 Tweets with human-rated sentiment labels (positive, negative, or neutral).


In [1]:
# importing the relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV